In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "./train/"
categories = ["1", "2", "3", "4", "5", "6", "7","8","9","10","11","12"]
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./numpy_data/multi_image_data.npy", xy)

print("ok", len(y))

1  파일 길이 :  7500
1  :  ./train//1\1_0000.jpg
1  :  ./train//1\1_0700.jpg
1  :  ./train//1\1_1400.jpg


KeyboardInterrupt: 

In [2]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

X_train, X_test, y_train, y_test = np.load('./numpy_data/multi_image_data.npy', allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

Using TensorFlow backend.
(67500, 64, 64, 3)
67500


In [3]:
categories = ["1", "2", "3", "4", "5", "6", "7","8","9","10","11","12"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [5]:
model = Sequential()
model.add(Conv2D(64, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [6]:
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

훈련되지 않은 모델의 정확도:  8.31%


In [7]:
model.load_weights("model/model.h5")
loss,acc = model.evaluate(X_test, y_test, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

복원된 모델의 정확도: 91.76%


In [20]:
model.predict(X_test[:1,0:])

array([[1.3768237e-09, 9.9928838e-01, 6.9432321e-04, 3.6024195e-16,
        9.3672755e-15, 1.5293414e-14, 3.3961570e-11, 1.4334563e-10,
        1.1179971e-05, 3.1356068e-17, 6.1126029e-06, 2.5829897e-11]],
      dtype=float32)

In [ ]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, "C:\Users\kkho5\Desktop\project\SimilarCeleb\js\model.js)

In [31]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = './model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/multi_img_classification.model'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [30]:
history = model.fit(X_train, y_train, batch_size=100, epochs=50, validation_data=(X_test, y_test),validation_split=0.2, callbacks=[checkpoint, early_stopping])

Train on 67500 samples, validate on 22500 samples
Epoch 1/50
 4700/67500 [=>............................] - ETA: 5:09 - loss: 2.2145 - accuracy: 0.1921

KeyboardInterrupt: 

In [45]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
model_dir = './model'

model_path = model_dir + '/multi_img_classification.model'
model_path

'./model/multi_img_classification.model'

In [46]:
model = keras.models.load_model(model_path)
model

In [49]:
score = model.evaluate(X_test, y_test, verbose=0)
y = model.predict(X_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1359.3352131184895
Test accuracy: 0.24324443936347961


In [48]:
X_test.shape

(22500, 64, 64, 3)

In [2]:
from keras.models import model_from_json
json_file = open("./model (1).json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [7]:
loaded_model.load_weights("./model/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [8]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
score = loaded_model.evaluate(X_test,y_test,verbose=1)
score

22500/22500 [==============================] - 76s 3ms/step


[48.565039027235244, 0.08168888837099075]

In [29]:
y = loaded_model.predict(X_test[:1,0:])
y

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [28]:
X_test[:1,0:].shape

(1, 64, 64, 3)

In [1]:
import numpy as np
import os
# 랜덤시드 고정시키기
np.random.seed(5)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=10,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  zoom_range=[0.9, 1.3],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')

for idx in range(1,13):
    categori = f"./train/{idx}/"
    for catimg in os.listdir(categori):
        if len(os.listdir(categori)) > 7500:
            break
        img = load_img(categori+"/"+catimg)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        i = 0

        # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
        for batch in data_aug_gen.flow(x, batch_size=1, save_to_dir=categori, save_prefix='tri', save_format='jpg'):
            i += 1
            if i > 120: 
                break

Using TensorFlow backend.


7512
7392
7508
7503
7509
7525
7538
